# Part 1 : Web Scrapping

*   In this part we will scrappe INDEED web site using Beautiful Soup










In [2]:
import bs4
import lxml
import pandas
import urllib

from urllib import request

In [ ]:

from selenium import webdriver 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# delay for selenium web driver wait
DELAY = 30

# create selenium driver
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome('<<PATH TO chromedriver>>', options=chrome_options)

# iterate over pages
for page in range(0, 10, 1):
    
    # open web page
    driver.get(f'https://www.apec.fr/candidat/recherche-emploi.html/emploi?page={page}')
    
    # wait for element with class 'container-result' to be added
    container_result = WebDriverWait(driver, DELAY).until(EC.presence_of_element_located((By.CLASS_NAME, "container-result")))
    # scroll to container-result
    driver.execute_script("arguments[0].scrollIntoView();", container_result)
    # get source HTML of the container-result element
    source = container_result.get_attribute('innerHTML')
    # print source
    print(source)
    # here you can continue work with the source variable either using selenium API or using BeautifulSoup API:
    # soup = BeautifulSoup(source, "html.parser")

# quit webdriver    
driver.quit()

In [ ]:

# Skills & Place of Work
skill = input('Enter your Skill: ').strip()
place = input('Enter the location: ').strip()
no_of_pages = int(input('Enter the # of pages to scrape: '))

indeed_posts=[]

for page in range(no_of_pages):
    
    # Connecting to India Indeed
        url = 'https://www.indeed.co.in/jobs?q=' + skill + \
            '&l=' + place + '&sort=date' +'&start='+ str(page * 10)
        
        # Get request to indeed with headers above (you don't need headers!)
        response = requests.get(url, headers=headers)
        html = response.text

        # Scrapping the Web (you can use 'html' or 'lxml')
        soup = BeautifulSoup(html, 'lxml')

        # Outer Most Entry Point of HTML:
        outer_most_point=soup.find('div',attrs={'id': 'mosaic-provider-jobcards'})
        
        # "UL" lists where the data are stored:
        
        for i in outer_most_point.find('ul'):
            
        # Job Title:
        
            job_title=i.find('h2',{'class':'jobTitle jobTitle-color-purple jobTitle-newJob'})
#             print(job_title)
            if job_title != None:
                jobs=job_title.find('a').text

        # Company Name:
    
            if i.find('span',{'class':'companyName'}) != None:
                company=i.find('span',{'class':'companyName'}).text   
                
        # Links: these Href links will take us to full job description
        
            if i.find('a') != None:
                links=i.find('a',{'class':'jcs-JobTitle'})['href']
                
        # Salary if available:
        
            if i.find('div',{'class':'attribute_snippet'}) != None:
                salary=i.find('div',{'class':'attribute_snippet'}).text

            else:
                salary='No Salary'

        # Job Post Date:

            if i.find('span', attrs={'class': 'date'}) != None:
                post_date = i.find('span', attrs={'class': 'date'}).text

        # Put everything together in a list of lists for the default dictionary
                        
            indeed_posts.append([company,jobs,links,salary, post_date])
            
            
# put together in list

# (create a dictionary with keys and a list of values from above "indeed_posts")

indeed_dict_list=defaultdict(list)

# Fields for our DF 

indeed_spec=['Company','job','link','Salary','Job_Posted_Date']

## Exercice 1 : Scrap a table on a wikipedia page

We would like to display on a map the location of Summer Olympic Games since 1896. We will use a [Wikipedia page](https://fr.wikipedia.org/wiki/Jeux_olympiques) to scrap the associated table.
Below is the code to extract the content of the page using `request` and display its title using `BeautifulSoup`.

In [97]:
jo = "https://fr.wikipedia.org/wiki/Jeux_olympiques"

request_text = request.urlopen(jo).read()
print(request_text[:1000])
print("\n")
page = bs4.BeautifulSoup(request_text, "lxml")
print(page.find("title"))

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" lang="fr" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Jeux olympiques \xe2\x80\x94 Wikip\xc3\xa9dia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 ve

Our objective here is to extract the different information in the first table "Jeux olympiques d'été" and to build a data frame.

To proceed, you will have to follow these steps:
*   Find the table
*   Collect each line of the table
*   Retrieve the different columns and transform them into text format. Also, use `strip` to format the value into a proper text format (e.g., without useless spaces). Store these lines (formated as a table of columns) in a table.

**Warning: We have two location for 1940. You have therefore to manage this case: identify the attribute indicating this fact and write a code to collect those lines and insert in the previous table in the right place**

* Collect headers of the HTML table
* Build a data frame from the result table and the headers








In [116]:
#[student]
tableau = page.find_all('table')[1]
print(tableau.stripped_strings)

<generator object Tag.stripped_strings at 0x7f61373299e0>


In [117]:
tableau = page.find('table', {'class': 'wikitable center'})
print(tableau)

<table class="wikitable center">
<caption><a href="/wiki/Jeux_olympiques_d%27%C3%A9t%C3%A9" title="Jeux olympiques d'été">Jeux olympiques d'été</a>
</caption>
<tbody><tr>
<th scope="col">Année
</th>
<th scope="col">Olympiade<br/><small>(Nº de l’<a href="/wiki/Olympiade" title="Olympiade">olympiade</a><sup class="reference" id="cite_ref-25"><a href="#cite_note-25"><span class="cite_crochet">[</span>NB 1<span class="cite_crochet">]</span></a></sup>)</small>
</th>
<th scope="col">Ville hôte
</th>
<th scope="col">Pays
</th>
<th scope="col">Continent
</th></tr>
<tr>
<td>1896</td>
<td align="center"><a href="/wiki/Jeux_olympiques_de_1896" title="Jeux olympiques de 1896">I</a></td>
<td><a href="/wiki/Ath%C3%A8nes" title="Athènes">Athènes</a> (1)</td>
<td><span class="flagicon"><span class="mw-image-border noviewer" typeof="mw:File"><a class="mw-file-description" href="/wiki/Fichier:Flag_of_Greece_(1822-1978).svg" title="Drapeau : Grèce"><img alt="" class="mw-file-element" data-file-height="60

In [118]:
lignes = tableau.find_all('tr')
dictionnaire_jo = {}
for s in lignes[0].find_all('th'):
    dictionnaire_jo[s.get_text(strip=True)]=[]
cles = list(dictionnaire_jo.keys())
nombre_de_colonne =len(cles)

for l in lignes[1:] :
    colonnes =  l.find_all('td')
    nb_col = len(colonnes) #Colonnes recupérées
    swap = nombre_de_colonne - nb_col
    if nb_col < nombre_de_colonne :
        for i in range(nb_col):
            dictionnaire_jo[cles[i+swap]].append(colonnes[i].get_text(strip=True))
        for s in range(swap) :
            dictionnaire_jo[cles[s]].append(dictionnaire_jo[cles[s]][-1])
    else :
        for s in range(len(cles)) :
            dictionnaire_jo[cles[s]].append(colonnes[s].get_text(strip=True))



In [119]:
# colonnes = tableau.find_all('td')
# print(colonnes)
df_jo = pandas.DataFrame(dictionnaire_jo)
df_jo

,Année,Olympiade(Nº de l’olympiade[NB 1]),Ville hôte,Pays,Continent
0,1896,I,Athènes(1),Grèce(1),Europe(1)
1,1900,II,Paris(1),France(1),Europe(2)
2,1904,III,Saint-Louis(1),États-Unis(1),Amérique du Nord(1)
3,1906,•(Intercalaires[NB 2]),Athènes(2),Grèce(2),Europe
4,1908,IV,Londres(1),Royaume-Uni(1),Europe(3)
5,1912,V,Stockholm(1),Suède(1),Europe(4)
6,1916,VI(annulés[NB 3]),Berlin,Allemagne,Europe
7,1920,VII,Anvers(1),Belgique(1),Europe(5)
8,1924,VIII,Paris(2),France(2),Europe(6)
9,1928,IX,Amsterdam(1),Pays-Bas(1),Europe(7)


## Exercice 2: Locate places on a map

The objective here is to identify organizer cities on a map.
You will have to code the following steps:
* Collect the URL of each city
* Go the this page using `urllib.request`
* Find the coordinates
* Store cities and coordinates in a data frame

In [135]:
#[student]
indice_ville = 2
urls = []
dic_villes = {}
for l in lignes[1:-1] :
    colonnes =  l.find_all('td')
    # colonne_ville =  l.find_all('td')[indice_ville]
    if len(colonnes) != nombre_de_colonne :
        if(colonnes[0].find('a').text not in dic_villes):
            dic_villes[colonnes[0].find('a').text] = colonnes[0].find('a')['href']
    else:
        if(colonnes[indice_ville].find('a').text not in dic_villes):
            dic_villes[colonnes[indice_ville].find('a').text] = colonnes[indice_ville].find('a')['href']
    # urls.append(colonne_ville.find('a'))
    # nb_col = len(colonnes) #Colonnes recupérées
    # for s in range(len(cles)) :
    #     dictionnaire_jo[cles[s]].append(colonnes[s].get_text(strip=True))
dic_villes

{'Athènes': '/wiki/Ath%C3%A8nes',
 'Paris': '/wiki/Paris',
 'Saint-Louis': '/wiki/Saint-Louis_(Missouri)',
 'Londres': '/wiki/Londres',
 'Stockholm': '/wiki/Stockholm',
 'Berlin': '/wiki/Berlin',
 'Anvers': '/wiki/Anvers',
 'Amsterdam': '/wiki/Amsterdam',
 'Los Angeles': '/wiki/Los_Angeles',
 'Tokyo': '/wiki/Tokyo',
 'Helsinki': '/wiki/Helsinki',
 'Melbourne': '/wiki/Melbourne',
 'Rome': '/wiki/Rome',
 'Mexico': '/wiki/Mexico',
 'Munich': '/wiki/Munich',
 'Montréal': '/wiki/Montr%C3%A9al',
 'Moscou': '/wiki/Moscou',
 'Séoul': '/wiki/S%C3%A9oul',
 'Barcelone': '/wiki/Barcelone',
 'Atlanta': '/wiki/Atlanta',
 'Sydney': '/wiki/Sydney',
 'Pékin': '/wiki/P%C3%A9kin',
 'Rio de Janeiro': '/wiki/Rio_de_Janeiro',
 'Paris ': '/wiki/Paris',
 'Brisbane': '/wiki/Brisbane'}

In [136]:
df_jo_villes = pandas.DataFrame(dic_villes)
df_jo_villes

ValueError: If using all scalar values, you must pass an index

We then transform degree coordinates into numerical coordinates to build a map


In [ ]:
import re

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S', 'O'):
        dd *= -1
    return dd

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    #potentially needs an adaptation
    #[student]
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])
    #lng = dms2dd(parts[4], parts[5], parts[6], parts[7])
    return lat

data['latitude'] = data['latitude'].apply(parse_dms)
#data['longitude'] = data['longitude'].apply(parse_dms)
print(data)

The map can be obtain with the following code

In [ ]:
import geopandas as gpd
from pathlib import Path
import folium
from IPython.display import display

gdf = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.longitude, data.latitude))

Path("leaflet").mkdir(parents=True, exist_ok=True)

center = gdf[['latitude', 'longitude']].mean().values.tolist()
sw = gdf[['latitude', 'longitude']].min().values.tolist()
ne = gdf[['latitude', 'longitude']].max().values.tolist()

m = folium.Map(location = center, tiles='openstreetmap')

# I can add marker one by one on the map
for i in range(0,len(gdf)):
    folium.Marker([gdf.iloc[i]['latitude'], gdf.iloc[i]['longitude']], popup=gdf.iloc[i]['name']).add_to(m)

m.fit_bounds([sw, ne])
display(m)

# Project

The objective of this project is to practice all concepts taught in the main lecture. Therefore, you will have to collect data around a thematic, identify a problematic, clean and format the data, provide some exploratory analysis and visualization, build models and evaluate them. You will have also to design dashboard and to storytell the whole pipeline.


The whole project will have two outputs describing the methodology and the results:
* A technical report targeting your datascientist colleagues
* An oral presentation targeting your CEO, chief, client. In this case, we assume that the audience is not specialized in data science. (But you also need to present the methodology to convince them to trust the results).


**Requests:**
* Team work of two people (same group throughout the semester)
* All your work should be stored on a git repo: [tutorial](https://github.com/baskiotisn/2IN013robot2023/blob/d979333fb80c9b6acd9515aaec040943d10d365c/docs/tutoriel_git.pdf)

**Remarks:**
There are also other libraries or issues you'll encounter when collecting data. Some of these are listed below, but don't be shy and interact with a search engine to solve your own issue!
* [Regular expressions](https://docs.python.org/3/howto/regex.html) might be useful!
* [API with authentication](https://www.geeksforgeeks.org/authentication-using-python-requests/)
*   [Selenium](https://selenium-python.readthedocs.io/) when pages are generated wia javascript scripts
* [Playwright](https://playwright.dev/) --> looks easier and more adapted than Selenium
* [Scrapy](https://scrapy.org/) for web crawling / or when you don't know the URL.[Tuto here](https://doc.scrapy.org/en/latest/intro/tutorial.html)
* [Summary of some difficulties in scrapping data from the web](https://www.zenrows.com/blog/web-scraping-challenges#page-structure-changes)

## Your daily task
Collecting data from the web and open data portals. Open data might include csv files, you can use them, but be aware that scraping should be your main acitivity in the dataset gathering.
You can begin to format the data, merge them to build dataframe that would be analyzed next week.